In [58]:
import requests
import json
import time

url = "https://nypdonline.org/api/reports/b805fa11-d5d2-43f7-8c23-1649f5d387f1/data"
headers = {
    "Accept": "application/json",
}

response = requests.get(url, headers=headers)
data = response.json()

# Explore the keys (optional, helps understand structure)
print(data.keys())

# Get the officer rows (usually inside 'data' or similar key)
rows = data.get("data", [])

# Print each officer row
for officer in rows:
    print(officer)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [49]:
import requests

all_data = []

url = "https://nypdonline.org/api/reports/b805fa11-d5d2-43f7-8c23-1649f5d387f1/data"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    payload = [
        {"key": "@SearchName", "values": [""]},
        {"key": "@LastNameFirstLetter", "values": [letter]}
    ]

    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        data = response.json()
        all_data.extend(data) 

        print(f"Fetching letter: {letter}")
        print(f"Yup Got {len(data)} officers for {letter}")
    
    else:
        print(f"❌ Failed on {letter} — Status: {response.status_code}")


Fetching letter: A
Yup Got 2070 officers for A
Fetching letter: B
Yup Got 2383 officers for B
Fetching letter: C
Yup Got 3298 officers for C
Fetching letter: D
Yup Got 2014 officers for D
Fetching letter: E
Yup Got 587 officers for E
Fetching letter: F
Yup Got 1316 officers for F
Fetching letter: G
Yup Got 2141 officers for G
Fetching letter: H
Yup Got 1580 officers for H
Fetching letter: I
Yup Got 320 officers for I
Fetching letter: J
Yup Got 851 officers for J
Fetching letter: K
Yup Got 1178 officers for K
Fetching letter: L
Yup Got 1861 officers for L
Fetching letter: M
Yup Got 3710 officers for M
Fetching letter: N
Yup Got 731 officers for N
Fetching letter: O
Yup Got 702 officers for O
Fetching letter: P
Yup Got 2056 officers for P
Fetching letter: Q
Yup Got 150 officers for Q
Fetching letter: R
Yup Got 2580 officers for R
Fetching letter: S
Yup Got 3121 officers for S
Fetching letter: T
Yup Got 1242 officers for T
Fetching letter: U
Yup Got 157 officers for U
Fetching letter: V
Y

In [44]:
all_data

[{'filterKey': '@TAXID',
  'filterRowValue': '959433',
  'columns': [{'columnTemplate': 'AANOUZ, ABDELHADI                                                                                       ',
    'value': 'AANOUZ, ABDELHADI                                                                                       '},
   {'value': 'DETECTIVE SPECIALIST'},
   {'value': '5664    '},
   {'value': '10/7/2015 12:00:00 AM'},
   {'value': '104'},
   {'value': '0'}]},
 {'filterKey': '@TAXID',
  'filterRowValue': '975305',
  'columns': [{'columnTemplate': 'AARONS, CAMELIA M                                                                                       ',
    'value': 'AARONS, CAMELIA M                                                                                       '},
   {'value': 'POLICE OFFICER'},
   {'value': '29755   '},
   {'value': '10/19/2022 12:00:00 AM'},
   {'value': '27'},
   {'value': '0'}]},
 {'filterKey': '@TAXID',
  'filterRowValue': '941300',
  'columns': [{'columnTem

In [51]:
cleaned_rows = []

for officer in all_data:  # assuming 'data' is your list of officers
    cols = officer["columns"]
    row = {
        "TAXID": officer["filterRowValue"],
        "Name": cols[0]["value"].strip(),
        "Rank": cols[1]["value"].strip(),
        "Shield": cols[2]["value"].strip(),
        "Hire Date": cols[3]["value"].strip(),
        "Arrests Total": cols[4]["value"].strip(),
        "Recognitions": cols[5]["value"].strip()
    }
    cleaned_rows.append(row)

In [52]:
import pandas as pd
df = pd.DataFrame(cleaned_rows)

In [78]:
df

,TAXID,Name,Rank,Shield,Hire Date,Arrests Total,Recognitions
0,959433,"AANOUZ, ABDELHADI",DETECTIVE SPECIALIST,5664,10/7/2015 12:00:00 AM,104,0
1,975305,"AARONS, CAMELIA M",POLICE OFFICER,29755,10/19/2022 12:00:00 AM,27,0
2,941300,"AARONSON, JACOB J",POLICE OFFICER,44,7/10/2006 12:00:00 AM,154,5
3,982069,"ABABIO, MALIK K",POLICE OFFICER,,4/30/2025 12:00:00 AM,0,0
4,972490,"ABAD, BRENDALIZ",POLICE OFFICER,5985,10/12/2021 12:00:00 AM,0,0
...,...,...,...,...,...,...,...
36668,954443,"ZVONIK, SERGEY A",POLICE OFFICER,30494,1/9/2013 12:00:00 AM,59,0
36669,933512,"ZWEBEN, HOWARD E",LIEUTENANT,,7/1/2003 12:00:00 AM,127,34
36670,961483,"ZWEIFLER, MICHAEL G",DETECTIVE SPECIALIST,4650,1/6/2016 12:00:00 AM,135,52
36671,935452,"ZWILLING, CHARLES J",SERGEANT,2777,7/1/2004 12:00:00 AM,227,3


In [54]:
df.to_csv("nypd_officers_full.csv", index=False)

Now for the extra information i gotta scrape

In [ ]:
detail_url = "https://nypdonline.org/api/reports/e59337f8-014e-46a0-af99-7b6a3ef4dfa3/data"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

profile_data = []

for i, officer in df.iterrows():
    taxid = df.loc[i, "TAXID"]  

    payload = [
        {"key": "@TAXID", "values": [str(taxid)]}
    ]

    try:
        response = requests.post(detail_url, headers=headers, json=payload)
        if response.status_code == 200:
            detail = response.json()

            # 👇 You'll need to explore the JSON structure here:
            profile_data.append({
                "TAXID": taxid,
                "Awards": ...,             # fill this based on structure
                "Training Summary": ...,   # fill this based on structure
                "Arrests Processed": ...,  # etc.
            })
        else:
            print(f"❌ {taxid} failed: status {response.status_code}")
            profile_data.append({"TAXID": taxid, "Error": f"Status {response.status_code}"})

    except Exception as e:
        print(f"🔥 Exception on {taxid}: {e}")
        profile_data.append({"TAXID": taxid, "Error": str(e)})

    time.sleep(0.5)  # To avoid hammering the server)

KeyboardInterrupt: 

In [77]:
import requests
import json

url = "https://nypdonline.org/api/reports/e59337f8-014e-46a0-af99-7b6a3ef4dfa3/data"
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

payload = [
    {"key": "@TAXID", "values": ["941300"]}
]

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(json.dumps(response.json(), indent=2))

200
[
  {
    "filterKey": "@TAXID",
    "columns": [
      {
        "value": "11/14/2022 12:00:00 AM"
      },
      {
        "value": "0"
      },
      {
        "filterValue": "ALLEG941300",
        "value": "1"
      }
    ]
  }
]


In [80]:
headers = {
    "Content-Type": "application/json",
    "Referer": "https://nypdonline.org/link/officer-profile"
}

tabs = {
    "Arrests Processed": "b9f649fc-b33f-47b2-9d7e-a7760090ae39",
    "Recognition and Awards": "916ef68f-3d47-46be-8e8a-79567a3e61f9",
    "Disciplinary History": "e59337f8-014e-46a0-af99-7b6a3ef4dfa3"
}

detailed_data = []

for i, row in df.iterrows():
    taxid = str(row["TAXID"])
    officer_details = {"TAXID": taxid}

    for tab, tab_id in tabs.items():
        url = f"https://nypdonline.org/api/reports/{tab_id}/data"
        payload = [{"key": "@TAXID", "values": [taxid]}]

        try:
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code == 200:
                data = response.json()

                # You can extract specific fields here:
                officer_details[f"{tab} Count"] = len(data)
            else:
                officer_details[f"{tab} Count"] = f"ERR:{response.status_code}"

        except Exception as e:
            officer_details[f"{tab} Count"] = f"EXC:{str(e)}"

        time.sleep(0.3)  # polite scraping

    detailed_data.append(officer_details)

KeyboardInterrupt: 

In [ ]:
detail_df = pd.DataFrame(detailed_data)

# Merge with original
full_df = df.merge(detail_df, on="TAXID", how="left")
hbop12erd
# Save
full_df.to_csv("nypd_officers_extra.csv", index=False